In [183]:
from pathlib import Path
from keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import joblib

In [133]:
# Transfer learning
# Build an image recognition system can identify picturs of tumor:
# 1- build a feature extractor: extract training features from our images
# training data: 1- tumors 2- no tumors  # 64x64 imagenet data
# write a code that use the pretrained model to extract features from our training images and save those features to a file

In [3]:
# load data - data has training (tumor,no-tumor) and test (tumor,no-tumor) datasets

zip_file_path=("/content/MRIBrainTumor.zip")
extracted_dir=("/content/extracted_data")
!mkdir -p $extracted_dir
with zipfile.ZipFile(zip_file_path,'r') as zip_ref:
  zip_ref.extractall(extracted_dir)
!ls $extracted_dir

Testing  Training


In [192]:
from pathlib import Path
from keras.preprocessing import image
import numpy as np

# path_to_no_tumor_directory and "path_to_pituitary_tumor_directory"
no_tumor_path = Path("/content/extracted_data/Training/no_tumor")
pituitary_tumor_path = Path("/content/extracted_data/Training/pituitary_tumor")

images = []
labels = []

for img_path in no_tumor_path.glob("*jpg"):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    images.append(img_array)
    labels.append(0)

for img_path in pituitary_tumor_path.glob("*jpg"):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    images.append(img_array)
    labels.append(1)

xtrain = np.array(images)
ytrain = np.array(labels)


In [193]:
ytrain.shape #(1222,)
xtrain.shape  #(1222, 224, 224, 3)

(1222, 224, 224, 3)

In [194]:
# Train-test split
x_train, x_test, y_train, y_test = train_test_split(xtrain, ytrain, random_state=10, test_size=0.2)

y_test.shape #(245,)
x_test.shape  #(245, 224, 224, 3)
x_train.shape  #(977, 224, 224, 3)
y_train.shape #(977,)

(977,)

In [195]:
# normalize image data to 0-1 range
x_train=vgg16.preprocess_input(x_train)

# load pretrained nn to useas a feature extractor
# create new vgg16 object: pretrained on image net - delete last layer (top) of nn - traing images: size images 64x64: 3 channels
pretrained_nn=vgg16.VGG16(weights="imagenet",include_top=False, input_shape=(224,224,3))

# feed all trainign images through the nn and capture results: predict function
# extract features for each image (all in one pass)
# f_x array will now contain the set of features that represent each of the training images in our dataset
features_x=pretrained_nn.predict(x_train)


31/31 [==============================] - 4s 115ms/step


In [196]:
#save the array of extracted features to a file (disc) - dump : writing an array to disc
joblib.dump(features_x,"x_train.dat")

#save the matching array of expecte values to a file
joblib.dump(y_train,"y_train.dat")


['y_train.dat']

In [197]:
# Train new Neural Network to use those extracted features
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib

#load datasets #load features that we extracted with the pre train vgg 16 nn
x_train=joblib.load("x_train.dat")
y_train=joblib.load("y_train.dat")

model=Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))  # not have cnv layers #has final dense layers
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

#compile
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [198]:
model.fit(x_train,y_train,epochs=3,shuffle=True,validation_split=0.1)



Epoch 1/3
28/28 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 0.3288 - val_loss: 0.0000e+00 - val_accuracy: 0.2857
Epoch 2/3
28/28 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.3254 - val_loss: 0.0000e+00 - val_accuracy: 0.2857
Epoch 3/3
28/28 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.3254 - val_loss: 0.0000e+00 - val_accuracy: 0.2857


In [199]:
model_structure=model.to_json()
f=Path("model_structure.json")
f.write_text(model_structure)
model.save_weights("model.weights.h5")

In [214]:
# Prediction
class_labels=["tumor","no tumor"]
f=Path("model_structure.json")
model_structure=f.read_text()
model=model_from_json(model_structure)
model.load_weights("model.weights.h5")

img=image.load_img("/content/brain.jpg", target_size=(224,224))
# 3D numpy array -> nn
image_array=image.img_to_array(img)

In [216]:
# batch of images as once- creare batches even 1 img
# batches as 4D array  1: list of images,  3:imae data
# 4D add new axis to it with numpy: np.expand_dims()
images=np.expand_dims(image_array,axis=0) #axis=0 :new axis is the 1st dimension

# normalize
images=vgg16.preprocess_input(images)

# use pretrained nn to extract features from our test img
feature_extraction_model=vgg16.VGG16(weights="imagenet",include_top=False,  input_shape=(224,224,3))
features=feature_extraction_model.predict(images)

#given the extracted features , make a final prediction using our own nn model
results=model.predict(features)

# since we are only testing 1 img with possible classs, we need to check only the 1st result
single_result=results[0][0]

print("Likelihood that this image contains a tumor: {}%".format(int(single_result*100)))
# 0% means it is not a tumor



1/1 [==============================] - 0s 27ms/step
Likelihood that this image contains a tumor: 0%
